<a href="https://colab.research.google.com/github/jess789550/machinelearning/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-11-25 14:49:47--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.2’

book-crossings.zip. 100%[===================>]  24.88M   107MB/s    in 0.2s    

2024-11-25 14:49:48 (107 MB/s) - ‘book-crossings.zip.2’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace BX-Books.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace BX-Users.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

print(df_books.head())
print(df_ratings.head())

         isbn                                              title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

                 author  
0    Mark P. O. Morford  
1  Richard Bruce Wright  
2          Carlo D'Este  
3      Gina Bari Kolata  
4       E. J. W. Barber  
     user        isbn  rating
0  276725  034545104X     0.0
1  276726  0155061224     5.0
2  276727  0446520802     0.0
3  276729  052165615X     3.0
4  276729  0521795028     6.0


In [4]:
# add your code here - consider creating a new cell for each section of code

# https://scikit-learn.org/dev/modules/generated/sklearn.neighbors.NearestNeighbors.html

# df_merge = pd.merge(df_books, df_ratings, on='isbn')
# neigh = NearestNeighbors(n_neighbors=2, radius=0.4)
# neigh.fit(df_merge.loc[:,"rating"]) # ERROR
# NearestNeighbors(...)
# neigh.kneighbors([[0, 0, 1.3]], 2, return_distance=False)
# nbrs = neigh.radius_neighbors(
#    [[0, 0, 1.3]], 0.4, return_distance=False
# )
# np.asarray(nbrs[0][0])

##################################################################################

# DeepAI - crashes & uses up RAM

# # Merge books and ratings
# df_merged = pd.merge(df_books, df_ratings, on='isbn', how='inner')

# # Prepare a pivot table - we can use user ratings for books
# pivot_table = df_merged.pivot_table(index='title', columns='user', values='rating').fillna(0)

# # Fit Nearest Neighbors model
# model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=6)  # n_neighbors = 6 to include the book itself
# model.fit(pivot_table.values)

####################################################################################

# https://github.com/zakaria-narjis/Book-Recommendation-Engine-using-KNN/blob/main/fcc_book_recommendation_knn.ipynb

df = df_ratings

# Number of ratings per user
counts1 = df['user'].value_counts()

# Number of ratings per IBSN number
counts2 = df['isbn'].value_counts()

# Filter out users who have provided less than 200 ratings
df = df[~df['user'].isin(counts1[counts1 < 200].index)]

# Filter out ISBN numbers (books) with less than 100 ratings
df = df[~df['isbn'].isin(counts2[counts2 < 100].index)]

# Merge data frames by ISBN column
df = pd.merge(right=df, left = df_books, on="isbn")

# Remove duplicates
df = df.drop_duplicates(["title", "user"])

# Create pivot table
piv = df.pivot(index='title', columns='user', values='rating').fillna(0)

# Extract the underlying NumPy array from the pivot table
matrix = piv.values
print(matrix.shape)
print(piv)

# Create nearest neighbours model and fit to data
model_knn=NearestNeighbors(metric='cosine',algorithm='brute')
model_knn.fit(matrix)

(673, 888)
user                                                254     2276    2766    \
title                                                                        
1984                                                   9.0     0.0     0.0   
1st to Die: A Novel                                    0.0     0.0     0.0   
2nd Chance                                             0.0    10.0     0.0   
4 Blondes                                              0.0     0.0     0.0   
A Beautiful Mind: The Life of Mathematical Geni...     0.0     0.0     0.0   
...                                                    ...     ...     ...   
Without Remorse                                        0.0     0.0     0.0   
Year of Wonders                                        0.0     0.0     0.0   
You Belong To Me                                       0.0     0.0     0.0   
Zen and the Art of Motorcycle Maintenance: An I...     0.0     0.0     0.0   
\O\" Is for Outlaw"                                  

NearestNeighbors(algorithm='brute', metric='cosine')

In [5]:
# function to return recommended books - this will be tested
# def get_recommends(book = ""):


#   return recommended_books

#################################################################################

# DeepAI
# def get_recommends(book_title):
#     if book_title not in pivot_table.index:
#         return f"Book '{book_title}' not found in the dataset."

#     # Find the index of the book title
#     book_index = pivot_table.index.get_loc(book_title)

#     # Find nearest neighbors
#     distances, indices = model.kneighbors(pivot_table.iloc[book_index, :].values.reshape(1, -1), n_neighbors=6)

#     # Compile a list of similar books excluding the input book itself
#     similar_books = []
#     for i in range(1, len(distances.flatten())):  # Start from 1 to skip the book itself
#         similar_books.append((pivot_table.index[indices.flatten()[i]], distances.flatten()[i]))

#     return similar_books

#################################################################################


# https://github.com/zakaria-narjis/Book-Recommendation-Engine-using-KNN/blob/main/fcc_book_recommendation_knn.ipynb
def get_recommends(book = ""):
  x=piv.loc[book].array.reshape(1, -1) # retrieves the row corresponding to the specified book from the pivot table
  distances,indices=model_knn.kneighbors(x,n_neighbors=6) # find the 6 nearest neighbors (including the book itself)
  R_books=[]
  for distance,indice in zip(distances[0],indices[0]): # distances to the nearest neighbors, and indices of those neighbors
    if distance!=0: # not original book
      R_book=piv.index[indice] # retrieves the book title using the index
      R_books.append([R_book,distance]) # append to list
  recommended_books=[book,R_books[::-1]] #  Since KNN finds the closest neighbors first (including the original input book), reversing the list provides a clean presentation of similar books, sorted from closest to furthest in terms of distance
  return recommended_books



In [6]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016210581447822], ['The Weight of Water', 0.7708583572697412], ['The Surgeon', 0.7699410973804288], ['I Know This Much Is True', 0.7677075092617776], ['The Lovely Bones: A Novel', 0.7234864549790632], ["Where the Heart Is (Oprah's Book Club (Paperback))", 2.220446049250313e-16]]]
You passed the challenge! 🎉🎉🎉🎉🎉
